In [12]:
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Input,GlobalMaxPool1D, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive//My Drive/Andmeteadus/Masinõpe/4N/")

Mounted at /content/drive


In [4]:
wordvec_loc = "word2vec_features_with_labels_after_removing_noisy_columns.csv"
w2v = pd.read_csv(wordvec_loc)

In [5]:
data_path = "cleaned_am_nam_articles.csv"
data = pd.read_csv(data_path)
data = data.dropna()
train, test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=True)
train_articles = train["article"]
train_labels = train["label"]
test_articles = test["article"]
test_labels = test["label"]

le = LabelEncoder()
le.fit(test_labels)
y_train = le.transform(train_labels)
y_test = le.transform(test_labels)

In [6]:
print("FOR TRAIN")
for i, el in enumerate(train_articles):
  if type(el) != str:
    print(i, el)
print("FOR TEST")
for i, el in enumerate(test_articles):
  if (type(el)) != str:
    print(i, el)
  if el == np.nan:
    print(i, el)

print(w2v.shape)

FOR TRAIN
FOR TEST
(1087, 2840)


In [9]:
vocabulary_size = 500
tokenizer = Tokenizer(num_words=vocabulary_size,oov_token="XXX") 
tokenizer.fit_on_texts(train_articles)
X_train_vec = tokenizer.texts_to_sequences(train_articles)
X_test_vec = tokenizer.texts_to_sequences(test_articles)
pikkus = 1000
X_train_pad = sequence.pad_sequences(X_train_vec, maxlen=pikkus)
X_test_pad = sequence.pad_sequences(X_test_vec, maxlen=pikkus)
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

checked_vocabulary_size = min(vocabulary_size, len(tokenizer.word_index)+1)
embedding_size = w2v.shape[0]
embedding_matrix = np.zeros((checked_vocabulary_size, embedding_size))
count = 0
count2 = 0
for word, i in tokenizer.word_index.items():
  if i>=vocabulary_size:
    continue
  try:
    count2 += 1
    embedding_matrix[i] = w2v[word]
  except KeyError:
    count += 1
    embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_size)

print(count, count2)
model = Sequential()
model.add(Embedding(checked_vocabulary_size,embedding_size,weights=[embedding_matrix],trainable=True))
model.add(LSTM(35, return_sequences=True, recurrent_dropout=0.2, dropout=0.1))
model.add(LSTM(35, return_sequences=False, recurrent_dropout=0.2, dropout=0.1))
model.add(Dense(2, activation='softmax'))

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

323 499
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 1087)        543500    
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 35)          157220    
_________________________________________________________________
lstm_5 (LSTM)                (None, 35)                9940      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 72        
Total params: 710,732
Trainable params: 710,732
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(X_test_pad.shape, y_test_cat.shape, X_train_pad.shape, y_train_cat.shape)

(217, 1000) (217, 2) (867, 1000) (867, 2)


In [10]:
model.fit(X_train_pad, y_train_cat,
          batch_size=64,
          epochs=5,
          validation_data=(X_test_pad, y_test_cat))

Epoch 1/5
14/14 [==============================] - 118s 8s/step - loss: 0.6863 - accuracy: 0.5536 - val_loss: 0.6670 - val_accuracy: 0.6359
Epoch 2/5
14/14 [==============================] - 117s 8s/step - loss: 0.6513 - accuracy: 0.6240 - val_loss: 0.6407 - val_accuracy: 0.6406
Epoch 3/5
14/14 [==============================] - 118s 8s/step - loss: 0.5877 - accuracy: 0.7093 - val_loss: 0.5933 - val_accuracy: 0.6912
Epoch 4/5
14/14 [==============================] - 117s 8s/step - loss: 0.5011 - accuracy: 0.7670 - val_loss: 0.5711 - val_accuracy: 0.6912
Epoch 5/5
14/14 [==============================] - 116s 8s/step - loss: 0.4154 - accuracy: 0.8281 - val_loss: 0.5445 - val_accuracy: 0.7097


In [14]:
y_pred_cat = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_cat, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

f1_score(y_true, y_pred)

0.6519337016574586

In [ ]:

def tootle_4(tekst):
  global tokenizer, pikkus, sequence
  toodeldudTekst = sequence.pad_sequences(tokenizer.texts_to_sequences([tekst]), maxlen=pikkus)
  return toodeldudTekst 
def ennusta_4(toodeldudTekst):
  global mudel4
  res = np.argmax(mudel4.predict(toodeldudTekst), axis=1)
  return res
